In [1]:
import sqlalchemy
from sqlalchemy import (create_engine, Column, String, Integer, Float, MetaData, 
                        Table, type_coerce, ForeignKey, case)
from sqlalchemy.orm import mapper, create_session, relationship, aliased, Session
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import case
import numpy as np
from sqlalchemy.ext.automap import automap_base
import matplotlib.pyplot as plt
import sqlalchemy.types as types
from sqlalchemy.sql import and_, or_, not_, func
from sqlalchemy.sql import select
import os
from os.path import isfile
import pandas as pd
import netCDF4 as nc
import datetime as dt
from salishsea_tools import evaltools as et, viz_tools
import datetime
import glob
import gsw
import pickle
import matplotlib as mpl
import cmocean
from scipy import stats as spst
import scipy.optimize as scopt

%matplotlib inline

In [2]:
# conversion factor
r_uE_W=1/0.2174 # (uE/m2/s) / (W/m2)

In [3]:
basedir='/data/eolson/results/MEOPAR/PARcalcs/'
dbname='DFO_CTD_withModelPAR.sqlite'
datelims=()
engine = create_engine('sqlite:///' + basedir + dbname, echo = False)
Base = automap_base()
# reflect the tables in salish.sqlite:
Base.prepare(engine, reflect=True)
# mapped classes have been created
# existing tables:
StationTBL=Base.classes.StationTBL
ObsTBL=Base.classes.ObsTBL
CalcsTBL=Base.classes.CalcsTBL
session = create_session(bind = engine, autocommit = False, autoflush = True)


In [4]:
session.query(ObsTBL.PAR1_units).distinct().all()

[(None), ('uE/m^2/sec')]

In [5]:
session.query(ObsTBL.PAR_1_units).distinct().all()

[(None), ('uE/m^2/sec')]

In [6]:
PAR=case([(ObsTBL.PAR!=None, ObsTBL.PAR)], else_=
             case([(ObsTBL.PAR1!=None, ObsTBL.PAR1)], else_=ObsTBL.PAR_1))

In [7]:
SA=case([(CalcsTBL.Salinity_T0_C0_SA!=None, CalcsTBL.Salinity_T0_C0_SA)], else_=
         case([(CalcsTBL.Salinity_T1_C1_SA!=None, CalcsTBL.Salinity_T1_C1_SA)], else_=
         case([(CalcsTBL.Salinity_SA!=None, CalcsTBL.Salinity_SA)], else_= None)))
CT=case([(CalcsTBL.Temperature_Primary_CT!=None, CalcsTBL.Temperature_Primary_CT)], else_=
         case([(CalcsTBL.Temperature_Secondary_CT!=None, CalcsTBL.Temperature_Secondary_CT)], else_=CalcsTBL.Temperature_CT))
ZD=case([(ObsTBL.Depth!=None,ObsTBL.Depth)], else_= CalcsTBL.Z)

In [8]:
qry=session.query(StationTBL.StartYear.label('Year'),StationTBL.StartMonth.label('Month'),
                StationTBL.StartDay.label('Day'),StationTBL.StartHour.label('Hour'),
                StationTBL.Lat,StationTBL.Lon,ZD.label('Z'),PAR.label('PAR'),SA.label('SA'),CT.label('CT')).\
        select_from(StationTBL).join(ObsTBL,ObsTBL.StationTBLID==StationTBL.ID).\
        join(CalcsTBL,CalcsTBL.ObsTBLID==ObsTBL.ID).filter(and_(StationTBL.Lat>47-3/2.5*(StationTBL.Lon+123.5),
                                            StationTBL.Lat<47-3/2.5*(StationTBL.Lon+121),
                                            StationTBL.Lat<51,
                                            StationTBL.Include==True,ObsTBL.Include==True,
                                            ObsTBL.PAR>=0))


In [9]:
qry2=session.query(StationTBL.StartYear.label('Year'),StationTBL.StartMonth.label('Month'),
                StationTBL.StartDay.label('Day'),StationTBL.StartHour.label('Hour'),
                StationTBL.Lat,StationTBL.Lon,func.min(ZD).label('minZ')).\
        select_from(StationTBL).join(ObsTBL,ObsTBL.StationTBLID==StationTBL.ID).\
        join(CalcsTBL,CalcsTBL.ObsTBLID==ObsTBL.ID).filter(and_(StationTBL.Lat>47-3/2.5*(StationTBL.Lon+123.5),
                                            StationTBL.Lat<47-3/2.5*(StationTBL.Lon+121),
                                            StationTBL.Lat<51,
                                            StationTBL.Include==True,ObsTBL.Include==True,
                                            ObsTBL.PAR>=0)).group_by(StationTBL.ID)
minzQ=session.query(StationTBL.ID,func.min(ZD).label('minZ')).\
        select_from(StationTBL).join(ObsTBL,ObsTBL.StationTBLID==StationTBL.ID).\
        join(CalcsTBL,CalcsTBL.ObsTBLID==ObsTBL.ID).filter(and_(StationTBL.Lat>47-3/2.5*(StationTBL.Lon+123.5),
                                            StationTBL.Lat<47-3/2.5*(StationTBL.Lon+121),
                                            StationTBL.Lat<51,
                                            StationTBL.Include==True,ObsTBL.Include==True,
                                            ObsTBL.PAR>=0)).group_by(StationTBL.ID)

In [10]:
df2=pd.DataFrame(qry2.all())


In [11]:
df3=pd.DataFrame(minzQ.all())

In [12]:
df1=pd.DataFrame(qry.all())

In [13]:
len(df2),len(df1),len(df3)

(1735, 319619, 1735)

In [14]:
subminzQ=minzQ.subquery()
subqry=qry.subquery()

In [15]:
sparQ=session.query(subminzQ.c.ID,subminzQ.c.minZ,subqry.c.PAR).select_from(subminzQ).join(subqry,and_(
    ObsTBL.)

SyntaxError: invalid syntax (<ipython-input-15-a29e58daa530>, line 2)

In [ ]:
df1=pd.DataFrame(qry.all())
df1['dtUTC']=[dt.datetime(int(y),int(m),int(d))+dt.timedelta(hours=h) for y,m,d,h in zip(df1['Year'],df1['Month'],df1['Day'],df1['Hour'])]


In [ ]:
len(df1)

In [ ]:
with nc.Dataset('/ocean/eolson/MEOPAR/NEMO-forcing/grid/mesh_mask201702_noLPE.nc') as mesh:
    tmask=np.copy(mesh.variables['tmask'][0,:,:,:])
    navlat=np.copy(mesh.variables['nav_lat'][:,:])
    navlon=np.copy(mesh.variables['nav_lon'][:,:])

In [ ]:
fig,ax=plt.subplots(1,1,figsize=(5,5))
ax.plot(df1['Lon'],df1['Lat'],'k.')
ax.contour(navlon,navlat,tmask[0,:,:],(0.5,))

In [ ]:
df1.head()

In [ ]:
df1.tail()

In [ ]:
flen=1
namfmt='nowcast'
#varmap={'N':'nitrate','Si':'silicon','Ammonium':'ammonium'}
filemap={'PAR':'carp_T','vosaline':'grid_T','votemper':'grid_T'}
#gridmap={'nitrate':'tmask','silicon':'tmask','ammonium':'tmask'}
fdict={'carp_T':1,'grid_T':1}
PATH='/results/SalishSea/nowcast-green.201812/'
saveloc='/data/eolson/results/MEOPAR/PARcalcs/PARCTD_HC201905.pkl'
if os.path.isfile(saveloc):
    data=pickle.load(open( saveloc, 'rb' ) )
else:
    data=et.matchData(df1,filemap,fdict,dt.datetime(2015,1,1),dt.datetime(2018,1,1),namfmt,PATH,flen)
    data.drop(data[data.k<0].index,inplace=True)
    data['mod_PAR']=data['mod_PAR']*r_uE_W # convert mod_PAR to muE/m2/s
    with open(saveloc,'wb') as f:
        pickle.dump(data,f)


In [ ]:
basedir='/data/eolson/results/MEOPAR/PARcalcs/'
dbname='DFOCTDwithModel.sqlite'
datelims=()
engine = create_engine('sqlite:///' + basedir + dbname, echo = False)
Base = automap_base()
# reflect the tables in salish.sqlite:
Base.prepare(engine, reflect=True)
# mapped classes have been created
# existing tables:
StationTBL=Base.classes.StationTBL
ObsTBL=Base.classes.ObsTBL
CalcsTBL=Base.classes.CalcsTBL
session = create_session(bind = engine, autocommit = False, autoflush = True)


In [ ]:
fig,ax=plt.subplots(1,1,figsize=(5,5))
ax.plot(data['PAR'],data['mod_PAR'],'k.',ms=1)

In [ ]:
ii=(data['mod_PAR']>=0)&(data['PAR']>=0)
fig,ax=plt.subplots(1,1,figsize=(5,5))
xx=data.loc[ii,['mod_PAR']].values
yy=data.loc[ii,['PAR']].values
ax.hist2d(xx[:,0],yy[:,0],bins=100,cmin=0,cmax=50);

In [ ]:
ii=(data['mod_PAR']>=0)&(data['PAR']>=0)
fig,ax=plt.subplots(1,1,figsize=(5,5))
yy=data.loc[ii,['mod_PAR']].values
xx=data.loc[ii,['PAR']].values
_,_,_,m=ax.hist2d(xx[:,0],yy[:,0],bins=100,norm=mpl.colors.LogNorm(), cmap=mpl.cm.gist_rainbow_r);
fig.colorbar(m,ax=ax)
ax.set_xlim(0,5000)
ax.set_ylim(0,5000)
ax.set_aspect(1)
ax.plot((0,5000),(0,5000),'k-')
ax.set_xlabel('obs')
ax.set_ylabel('model')
ax.set_title('PAR Direct Comparison: HC201812')

In [ ]:
ii=(data['mod_PAR']>=0)&(data['PAR']>=0)
fig,ax=plt.subplots(1,1,figsize=(5,5))
yy=data.loc[ii,['mod_PAR']].values
xx=data.loc[ii,['PAR']].values
_,_,_,m=ax.hist2d(np.log10(xx[:,0]+1),np.log10(yy[:,0]+1),bins=100,norm=mpl.colors.LogNorm(), cmap=mpl.cm.gist_rainbow_r);
fig.colorbar(m,ax=ax)
ax.set_xlim(0,4)
ax.set_ylim(0,4)
ax.set_aspect(1)
ax.plot((0,4),(0,4),'k-')
ax.set_xlabel('obs')
ax.set_ylabel('model')
ax.set_title('log10 PAR Direct Comparison: HC201812')

In [ ]:
data.keys()

In [ ]:
stas=data.groupby(by=['Year','Month','Day','Lat','Lon'])

In [ ]:
stas

In [ ]:
with nc.Dataset('/results/SalishSea/nowcast-green.201812/01jan16/SalishSea_1h_20160101_20160101_ptrc_T.nc') as ftemp:
    deptht=ftemp.variables['deptht'][:]

In [ ]:
deptht[5]

In [ ]:
data['mod_Z']=[deptht[ii] for ii in data['k']]

In [ ]:
fig,ax=plt.subplots(1,1,figsize=(5,5))
ax.plot(data['Z'],data['mod_Z'],'k.',ms=1)
ax.set_xlabel('Obs')
ax.set_ylabel('Model')

In [ ]:
ii=(data['mod_PAR']>=0)&(data['PAR']>=0)
ik=data.k<10
fig,ax=plt.subplots(1,1,figsize=(5,5))
yy=data.loc[ii&ik,['mod_PAR']].values
xx=data.loc[ii&ik,['PAR']].values
yz=data.loc[ii&ik,['mod_Z']].values
xz=data.loc[ii&ik,['Z']].values
_,_,_,m=ax.hist2d(np.log10(xx[:,0]+1),np.log10(yy[:,0]+1),bins=100,norm=mpl.colors.LogNorm(), cmap=mpl.cm.gist_rainbow_r);
fig.colorbar(m,ax=ax)
ax.set_xlim(0,4)
ax.set_ylim(0,4)
ax.set_aspect(1)
ax.plot((0,4),(0,4),'k-')
ax.set_xlabel('obs')
ax.set_ylabel('model')
ax.set_title('PAR Direct Comparison Upper 10m: HC201812')

In [ ]:
ii=(data['mod_PAR']>=0)&(data['PAR']>=0)
ik=data.k<10
fig,ax=plt.subplots(1,1,figsize=(5,5))
yy=data.loc[ii&ik,['mod_PAR']].values
xx=data.loc[ii&ik,['PAR']].values
yz=data.loc[ii&ik,['mod_Z']].values
xz=data.loc[ii&ik,['Z']].values
_,_,_,m=ax.hist2d(np.log10(xx[:,0]+1),np.log10(yy[:,0]+1),bins=100,norm=mpl.colors.LogNorm(), cmap=mpl.cm.gist_rainbow_r);
fig.colorbar(m,ax=ax)
ax.set_xlim(0,4)
ax.set_ylim(0,4)
ax.set_aspect(1)
ax.plot((0,4),(0,4),'k-')
ax.set_xlabel('obs')
ax.set_ylabel('model')
ax.set_title('PAR Bias vs depth: HC201812')

In [ ]:
plt.hist(yz-xz)
plt.title('hist: model z - obs z')

In [ ]:
fig,ax=plt.subplots(1,1,figsize=(5,5))
ax.plot(yz-xz,np.log10(yy[:,0]+1)-np.log10(xx[:,0]+1),'k.',ms=1)
ax.set_xlabel('Model Z - Obs Z')
ax.set_ylabel('log10(model PAR +1)-log10(obs PAR +1)')

In [ ]:
ii=(data['mod_PAR']>=0)&(data['PAR']>=0)
ik=data.k<2
fig,ax=plt.subplots(1,1,figsize=(5,5))
yy=data.loc[ii&ik,['mod_PAR']].values
xx=data.loc[ii&ik,['PAR']].values
yz=data.loc[ii&ik,['mod_Z']].values
xz=data.loc[ii&ik,['Z']].values
_,_,_,m=ax.hist2d(np.log10(xx[:,0]+1),np.log10(yy[:,0]+1),bins=100,norm=mpl.colors.LogNorm(), cmap=mpl.cm.gist_rainbow_r);
fig.colorbar(m,ax=ax)
ax.set_xlim(0,4)
ax.set_ylim(0,4)
ax.set_aspect(1)
ax.plot((0,4),(0,4),'k-')
ax.set_xlabel('obs')
ax.set_ylabel('model')
ax.set_title('PAR Direct Comparison Upper 2m: HC201812')

In [ ]:
data.keys()

In [ ]:
temp=data.loc[(data['PAR']>0)&(data['mod_PAR']==0)&(data['Z']<3),['dtUTC','Lat','Lon','Z','mod_Z','Hour']]

In [ ]:
temp

In [ ]:
fig,ax=plt.subplots(1,1,figsize=(16,1))
ax.plot(temp['dtUTC'],temp['Z'],'k.')

In [ ]:
plt.hist(temp['Hour'])

In [ ]:
with nc.Dataset('/ocean/eolson/MEOPAR/NEMO-forcing/grid/mesh_mask201702_noLPE.nc') as mesh:
    tmask=np.copy(mesh.variables['tmask'][0,:,:,:])
    navlat=np.copy(mesh.variables['nav_lat'][:,:])
    navlon=np.copy(mesh.variables['nav_lon'][:,:])

In [ ]:
fig,ax=plt.subplots(2,2,figsize=(12,12))
ax=ax.flatten()
for yr in range(2015,2018):
    dyr=data.loc[(data['Year']==yr)&(~np.isnan(data['Lon']))&(~np.isnan(data['Lat']))&~np.isnan(data['PAR'])&(data['mod_PAR']>=0)&(data['Z']<2)]
    ax[yr-2015].contour(navlon,navlat,tmask[0,:,:],(0.5,))
    ax[yr-2015].set_title(str(yr)+' log model -log obs Surf PAR')
    s,xb,yb,nb=spst.binned_statistic_2d(dyr['Lon'],dyr['Lat'],
                np.log10(dyr['mod_PAR']+1)-np.log10(dyr['PAR']+1), 
                statistic='mean', bins=20)
    m=ax[yr-2015].pcolormesh(xb,yb,s.T,cmap=cmocean.cm.balance,vmin=-1,vmax=1,shading='flat')
    plt.colorbar(m,ax=ax[yr-2015])
dyr=data.loc[(~np.isnan(data['Lon']))&(~np.isnan(data['Lat']))&~np.isnan(data['PAR'])&~np.isnan(data['mod_PAR'])&(data['Z']<2)]
yr=yr+1
ax[yr-2015].contour(navlon,navlat,tmask[0,:,:],(0.5,))
ax[yr-2015].set_title('All yrs model - obs Surf PAR')
s,xb,yb,nb=spst.binned_statistic_2d(dyr['Lon'],dyr['Lat'],
           np.log10(dyr['mod_PAR']+1)-np.log10(dyr['PAR']+1), 
            statistic='mean', bins=20)
m=ax[yr-2015].pcolormesh(xb,yb,s.T,cmap=cmocean.cm.balance,vmin=-1,vmax=1,shading='flat')
plt.colorbar(m,ax=ax[yr-2015])

In [ ]:
fig,ax=plt.subplots(2,2,figsize=(12,12))
ax=ax.flatten()
for yr in range(2015,2018):
    dyr=data.loc[(data['Year']==yr)&(~np.isnan(data['Lon']))&(~np.isnan(data['Lat']))&~np.isnan(data['PAR'])&(data['mod_PAR']>=0)&(data['Z']<2)]
    ax[yr-2015].contour(navlon,navlat,tmask[0,:,:],(0.5,))
    ax[yr-2015].set_title(str(yr)+' model - obs Surf PAR')
    s,xb,yb,nb=spst.binned_statistic_2d(dyr['Lon'],dyr['Lat'],
               dyr['mod_PAR']-dyr['PAR'], 
                statistic='mean', bins=20)
    m=ax[yr-2015].pcolormesh(xb,yb,s.T,cmap=cmocean.cm.balance,vmin=-1000,vmax=1000,shading='flat')
    plt.colorbar(m,ax=ax[yr-2015])
dyr=data.loc[(~np.isnan(data['Lon']))&(~np.isnan(data['Lat']))&~np.isnan(data['PAR'])&~np.isnan(data['mod_PAR'])&(data['Z']<2)]
yr=yr+1
ax[yr-2015].contour(navlon,navlat,tmask[0,:,:],(0.5,))
ax[yr-2015].set_title('All yrs model - obs Surf PAR')
s,xb,yb,nb=spst.binned_statistic_2d(dyr['Lon'],dyr['Lat'],
           dyr['mod_PAR']-dyr['PAR'], 
            statistic='mean', bins=20)
m=ax[yr-2015].pcolormesh(xb,yb,s.T,cmap=cmocean.cm.balance,vmin=-1000,vmax=1000,shading='flat')
plt.colorbar(m,ax=ax[yr-2015])

In [ ]:
fig,ax=plt.subplots(2,2,figsize=(12,12))
print('Where model or obs PAR>100 muE/m2/s')
ax=ax.flatten()
for yr in range(2015,2018):
    dyr=data.loc[(data['Year']==yr)&(~np.isnan(data['Lon']))&(~np.isnan(data['Lat']))&((data['PAR']>=100)|(data['mod_PAR']>=100))&(data['Z']<2)]
    ax[yr-2015].contour(navlon,navlat,tmask[0,:,:],(0.5,))
    ax[yr-2015].set_title(str(yr)+' model - obs Surf PAR')
    s,xb,yb,nb=spst.binned_statistic_2d(dyr['Lon'],dyr['Lat'],
                dyr['mod_PAR']-dyr['PAR'], 
                statistic='mean', bins=20)
    m=ax[yr-2015].pcolormesh(xb,yb,s.T,cmap=cmocean.cm.balance,vmin=-1000,vmax=1000,shading='flat')
    plt.colorbar(m,ax=ax[yr-2015])
dyr=data.loc[(~np.isnan(data['Lon']))&(~np.isnan(data['Lat']))&((data['PAR']>=100)|(data['mod_PAR']>=100))&(data['Z']<2)]
yr=yr+1
ax[yr-2015].contour(navlon,navlat,tmask[0,:,:],(0.5,))
ax[yr-2015].set_title('All yrs model - obs Surf PAR')
s,xb,yb,nb=spst.binned_statistic_2d(dyr['Lon'],dyr['Lat'],
           dyr['mod_PAR']-dyr['PAR'], 
            statistic='mean', bins=20)
m=ax[yr-2015].pcolormesh(xb,yb,s.T,cmap=cmocean.cm.balance,vmin=-1000,vmax=1000,shading='flat')
plt.colorbar(m,ax=ax[yr-2015])

In [ ]:
fig,ax=plt.subplots(2,2,figsize=(12,12))
print('Where model or obs PAR>100 muE/m2/s')
ax=ax.flatten()
for yr in range(2015,2018):
    dyr=data.loc[(data['Year']==yr)&(~np.isnan(data['Lon']))&(~np.isnan(data['Lat']))&((data['PAR']>=100)|(data['mod_PAR']>=100))&(data['Z']<6)&(data['Z']>4)]
    ax[yr-2015].contour(navlon,navlat,tmask[0,:,:],(0.5,))
    ax[yr-2015].set_title(str(yr)+' model - obs 4-6m PAR')
    s,xb,yb,nb=spst.binned_statistic_2d(dyr['Lon'],dyr['Lat'],
                dyr['mod_PAR']-dyr['PAR'], 
                statistic='mean', bins=20)
    m=ax[yr-2015].pcolormesh(xb,yb,s.T,cmap=cmocean.cm.balance,vmin=-1000,vmax=1000,shading='flat')
    plt.colorbar(m,ax=ax[yr-2015])
dyr=data.loc[(~np.isnan(data['Lon']))&(~np.isnan(data['Lat']))&((data['PAR']>=100)|(data['mod_PAR']>=100))&(data['Z']<6)&(data['Z']>4)]
yr=yr+1
ax[yr-2015].contour(navlon,navlat,tmask[0,:,:],(0.5,))
ax[yr-2015].set_title('All yrs model - obs 4-6m PAR')
s,xb,yb,nb=spst.binned_statistic_2d(dyr['Lon'],dyr['Lat'],
           dyr['mod_PAR']-dyr['PAR'], 
            statistic='mean', bins=20)
m=ax[yr-2015].pcolormesh(xb,yb,s.T,cmap=cmocean.cm.balance,vmin=-1000,vmax=1000,shading='flat')
plt.colorbar(m,ax=ax[yr-2015])

In [ ]:
np.max(data['mod_PAR'])

In [ ]:
1/0.2174

In [ ]:
data.keys()

In [ ]:
data['staFile2']=[ii[49:] for ii in data['staFile']]
data['obsFile2']=[ii[49:] for ii in data['obsFile']]

In [ ]:
data2=data.loc[(data.mod_PAR-data.PAR<-1000)&(data.Year==2017),['dtUTC','Lat','Lon','mod_PAR','PAR','staFile2','obsFile2']]

In [ ]:
fig,ax=plt.subplots(1,1,figsize=(5,5))
ax.contour(navlon,navlat,tmask[0,:,:],(0.5,))
ax.plot(data2.Lon,data2.Lat,'r*')

In [ ]:
data2.loc[(data2.Lat<49.8)&(data2.Lat>49.3)&(data2.Lon<-124.2)]